In [69]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import random as rd
import math
import glob

In [70]:
allFiles = glob.glob("data/*.csv")
df = pd.DataFrame()
list_ = []

for file_ in allFiles:
    df = pd.read_csv(file_, index_col=None, header=0, sep=',')
    list_.append(df)
    
df = pd.concat(list_)
df.head()

,1,2,3,4,5,6,7,8,9,10,...,24,25,26,27,28,29,30,31,32,Phase
0,-0.005009,-0.000964,0.000573,0.008623,0.005667,0.001302,-0.000631,0.000130,-0.000048,0.007762,...,1.880800e-04,0.005133,0.010400,0.000646,0.007871,0.004631,0.000963,0.000092,0.000438,D
1,0.004905,0.001209,-0.000649,0.004737,0.003166,0.000819,-0.000572,-0.000015,0.000023,0.002706,...,-7.500000e-07,0.005093,0.005756,0.000573,0.003459,0.000730,0.000332,0.000012,0.000433,D
2,-0.002393,-0.000216,0.000136,0.003028,0.001212,0.000336,-0.000449,0.000017,0.000047,0.002868,...,-3.920000e-05,0.002406,0.003279,0.000452,0.003261,0.002412,0.000852,0.000042,0.000202,D
3,-0.001394,-0.000242,0.000056,0.001182,0.000575,0.000225,-0.000479,-0.000050,0.000104,0.001171,...,-3.184000e-05,0.001416,0.001334,0.000493,0.001358,0.000313,0.000611,0.000029,0.000596,D
4,-0.000156,-0.000004,0.000023,0.001585,0.000630,0.000094,-0.000303,0.000097,0.000065,0.001579,...,-2.015000e-05,0.000158,0.001709,0.000325,0.001713,0.000203,0.000069,0.000038,0.000069,D


In [71]:
len(df)

9873

In [72]:
y=df['Phase'].values.tolist()
#type(y); y.describe(); y.head(3); y.shape; y.unique() # (165632, )
dic = {'D':0, 'P':1, 'S':2, 'H':3, 'R':4}

y = [dic.get(n, n) for n in y]


In [73]:
X=df.drop(labels=['Phase'], axis=1, inplace = True)
X = df.astype(float)
X = df.values
X

array([[-5.00942e-03, -9.63710e-04,  5.72770e-04, ...,  9.63100e-04,
         9.16400e-05,  4.38310e-04],
       [ 4.90517e-03,  1.20913e-03, -6.48600e-04, ...,  3.31860e-04,
         1.23000e-05,  4.32710e-04],
       [-2.39286e-03, -2.15880e-04,  1.36300e-04, ...,  8.51980e-04,
         4.17000e-05,  2.01530e-04],
       ...,
       [ 3.07408e-03,  7.87019e-03, -9.62140e-04, ...,  7.11700e-05,
         1.69731e-03,  1.11780e-04],
       [ 3.29680e-03,  8.46747e-03, -1.03479e-03, ...,  2.32600e-05,
         1.38730e-04,  1.13470e-04],
       [ 2.04320e-04, -4.04600e-05,  5.77500e-05, ...,  3.69200e-05,
         1.83985e-03,  3.59900e-05]])

In [81]:
class K_Means:
    def __init__(self, k=5, tol=0.001, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter

    def fit(self,data):

        self.centroids = {}

        for i in range(self.k):
            self.centroids[i] = data[i]

        for i in range(self.max_iter):
            self.classifications = {}

            for i in range(self.k):
                self.classifications[i] = []

            for featureset in data:
                distances = [np.linalg.norm(featureset-self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classifications[classification].append(featureset)

            prev_centroids = dict(self.centroids)

            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification],axis=0)

            optimized = True

            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/original_centroid*100.0) > self.tol:
                    #print(np.sum((current_centroid-original_centroid)/original_centroid*100.0))
                    optimized = False

            if optimized:
                break

    def predict(self,data):
        distances = [np.linalg.norm(data-self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification

In [84]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

6911
2962
6911
2962


In [85]:
clf = K_Means()
clf.fit(X_train)

correct = 0
for i in range(len(X_test)):
    predict_me = np.array(X_test[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = clf.predict(predict_me)
    print("Predict: ", prediction, " || Actual: ", y_test[i])
    if prediction == y_test[i]:
        correct += 1


print("Accuracy",correct/len(X_test))

Predict:  0  || Actual:  0
Predict:  3  || Actual:  2
Predict:  0  || Actual:  2
Predict:  0  || Actual:  0
Predict:  3  || Actual:  1
Predict:  0  || Actual:  2
Predict:  0  || Actual:  2
Predict:  0  || Actual:  0
Predict:  0  || Actual:  0
Predict:  0  || Actual:  2
Predict:  0  || Actual:  0
Predict:  0  || Actual:  0
Predict:  0  || Actual:  2
Predict:  0  || Actual:  0
Predict:  0  || Actual:  0
Predict:  4  || Actual:  4
Predict:  0  || Actual:  2
Predict:  0  || Actual:  0
Predict:  1  || Actual:  4
Predict:  2  || Actual:  2
Predict:  1  || Actual:  1
Predict:  1  || Actual:  2
Predict:  0  || Actual:  1
Predict:  1  || Actual:  4
Predict:  0  || Actual:  0
Predict:  0  || Actual:  0
Predict:  2  || Actual:  1
Predict:  0  || Actual:  2
Predict:  0  || Actual:  0
Predict:  4  || Actual:  2
Predict:  0  || Actual:  4
Predict:  0  || Actual:  3
Predict:  1  || Actual:  4
Predict:  0  || Actual:  0
Predict:  0  || Actual:  2
Predict:  0  || Actual:  1
Predict:  2  || Actual:  2
P